In [ ]:
# 2D CNN for tumor detection from input brain MRIs

import cv2
import os
import csv
import random
import numpy as np
import pandas as pd
import imageio
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import gc
import seaborn as sns
from sklearn.model_selection import train_test_split
from keras import layers
from keras import models
from keras import optimizers
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img
from sklearn.metrics import classification_report

nrows = 256
ncolumns = 256
channels = 1

path_in = "..."
dir_out = "..."
path_out = dir_out + "Run/"

if not os.path.exists(path_out):
    os.makedirs(path_out)
    print("Directory Created")
else:    
    print("Directory Already Exists") 

def process_images (img_list):
    X = [] # images
    y = [] # labels
    
    for img in img_list:
        X.append(cv2.imread(img))
        
        if 'Y' in img:
            y.append(1) # 1 = Tumor
        elif 'N' in img:
            y.append(0) # 0 = Non-tumor
    
    return X, y

train_dir = path_in + 'Train'
test_dir = path_in + 'Test'

train_imgs = ['Train/{}'.format(i) for i in os.listdir(train_dir)]
random.shuffle(train_imgs)

test_imgs = ['Test/{}'.format(i) for i in os.listdir(test_dir)]

#print(train_imgs)
#print(test_imgs)

# Get images and labels
X, y = process_images(train_imgs)

del train_imgs
gc.collect()

#print(X)
#print(y)

X = np.array(X)
y = np.array(y)


#print('Shape of train images : ', X.shape)
#print('Shape of train labels : ', y.shape)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.20, random_state = 2)

print('Shape of train images : ', X_train.shape)
print('Shape of train labels : ', y_train.shape)
print('Shape of validation images : ', X_val.shape)
print('Shape of validation labels : ', y_val.shape)

del X
del y
gc.collect()

ntrain = len(X_train)
nval = len(X_val)

batch_size = 32

train_datagen = ImageDataGenerator(rescale = 1./255)
val_datagen = ImageDataGenerator(rescale = 1./255)

model = models.Sequential()

# 1st Layer
model.add(layers.Conv2D(32,(8,8),activation='relu', input_shape=(256, 256, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))

# 2nd Layer
model.add(layers.Conv2D(64, (8, 8), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))

# Output Layer
model.add(layers.Flatten())
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss = 'binary_crossentropy', optimizer = optimizers.RMSprop(lr=1e-4), metrics = ['acc'])

train_generator = train_datagen.flow(X_train, y_train, batch_size = batch_size)
val_generator = val_datagen.flow(X_val, y_val, batch_size = batch_size)

# Training
history = model.fit_generator(train_generator,
                              steps_per_epoch = 10,
                              epochs = 50,
                              validation_data = val_generator,
                              validation_steps = 10)

# Save model
#model.save_weights(path_out + 'model_weights.h5')
#model.save(path_out + 'model_keras.h5')

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

avg_acc = np.mean(acc) * 100
avg_val_acc = np.mean(val_acc) * 100
avg_loss = np.mean(loss) * 100
avg_val_loss = np.mean(val_loss) * 100

file0 = open(path_out + "avg_eval.txt","w")#append mode 
file0.write("Average Training Accuracy: " + str(avg_acc)) 
file0.write("\n")
file0.write("Average Validation Accuracy: " + str(avg_val_acc)) 
file0.write("\n")
file0.write("Average Training Loss: " + str(avg_loss)) 
file0.write("\n")
file0.write("Average Validation Loss: " + str(avg_val_loss)) 
file0.write("\n")
file0.close()

np.savetxt(path_out + 'train_acc.txt', acc, delimiter="\n") 
np.savetxt(path_out + 'val_acc.txt', val_acc, delimiter="\n") 
np.savetxt(path_out + 'train_loss.txt', loss, delimiter="\n") 
np.savetxt(path_out + 'val_loss.txt', val_loss, delimiter="\n") 

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'b', label = 'Training Accuracy')
plt.plot(epochs, val_acc, 'r', label = 'Validation Accuracy')
plt.title('Training & Validation Accuracy')
plt.legend()
plt.savefig(path_out + 'Accuracy.jpg')
plt.show()

plt.plot(epochs, loss, 'b', label = 'Training Loss')
plt.plot(epochs, val_loss, 'r', label = 'Validation Loss')
plt.title('Training & Validation Loss')
plt.legend()
plt.savefig(path_out + 'Loss.jpg')
plt.show()

X_test, y_test = process_images(test_imgs)

num_test = len(X_test)

#print('Shape of test images : ', X_test.shape)
#print('Shape of test labels : ', y_test.shape)

np.savetxt(path_out + 'y_test.txt', y_test, delimiter="\n") 

x = np.array(X_test)

test_datagen = ImageDataGenerator(rescale = 1./255)

y_pred = []

p = 1

batch = test_datagen.flow(x)
pred = model.predict(batch)

test_eval = model.evaluate(x, y_test, verbose=0)
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

test_acc = test_eval[1] * 100
test_loss = test_eval[0] * 100

file_t = open(path_out + "test_eval.txt","w")#append mode 
file_t.write("Test Accuracy: " + str(test_acc)) 
file_t.write("\n")
file_t.write("Test Loss: " + str(test_loss)) 
file_t.close() 

np.savetxt(path_out + 'y_prediction.txt', pred, delimiter="\n") 

a = 0

for value in pred:
    if (pred[a] > 0.5):
        y_pred.append(1)
        a = a + 1
    else:
        y_pred.append(0)
        a = a + 1
        
np.savetxt(path_out + 'y_pred.txt', y_pred, delimiter="\n")

c = 0

for i in range(0, num_test)):
    a = y_test[i]
    b = y_pred[i]
    if(a==b):
        c = c + 1
    print('c: ', c)
    
pred_acc = (c/num_test)) * 100

file1 = open(path_out + "pred_acc.txt","w")#append mode 
file1.write("Correct Predictions: " + str(c)) 
file1.write("\n")
file1.write("Prediction Accuracy: " + str(pred_acc)) 
file1.close() 

print('Total images in Test set :', str(num_test))
print('Correctly predicted label: ', c)

print('Prediction Accuracy : ', pred_acc)

num_classes = 2

target_names = ["Class {}".format(i) for i in range(num_classes)]
print(classification_report(y_test, y_pred, target_names=target_names))
clsf_report = pd.DataFrame(classification_report(y_true = y_test, y_pred = y_pred, output_dict=True)).transpose()
clsf_report.to_csv(path_out + 'Classification Report.csv', index= True)